# Query a pre-built database of chess positions

This demo shows you how to search a pre-bulit database of chess positions for positions that are similar to your own query position.

### Enter your query

First you need to specify the position for which you want to find similar positions for. Use the widget in the following cell to set up a board and then press the *Export Position as White* button if it is White's turn and *Export Position as Black* button if it is Blacks's turn.

In [ ]:
%%html
<style>
.spare-pieces-7492f img {
    display: inline-block
}
</style>

<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous">

<div id="myBoard" style="width: 400px"></div>
<button id="startBtn">Start Position</button>
<button id="clearBtn">Clear Board</button>
<button id="exportWhite">Export Position as White</button>
<button id="exportBlack">Export Position as Black</button>

<script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script>

<script>
var board = Chessboard('myBoard', {
  draggable: true,
  dropOffBoard: 'trash',
  sparePieces: true
});
var fen = undefined;
document.getElementById('startBtn').onclick = board.start;
document.getElementById('clearBtn').onclick = board.clear;
document.getElementById('exportWhite').onclick = function(){
    var fen = board.fen();
    IPython.notebook.kernel.execute("fen='"+fen+" w'");
};
document.getElementById('exportBlack').onclick = function(){
    var fen = board.fen();
    IPython.notebook.kernel.execute("fen='"+fen+" b'");
};
</script>

In [ ]:
print(f"The fen string of your position is: {fen}")

**Alternatively** if the widget above does not work you can visit [lichess.org](https://lichess.org/editor) retrieve the fen string of your position there and then paste it in the cell below.

In [1]:
'''
Execute this cell if you don't want to use the widget above.
Replace the example fen string with your own.
'''
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

### Load the database of chess positions

In [2]:
import faiss
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from searchpos import *

Load the database into memory. *You need more than 1.6GB of RAM.*

In [3]:
filepath = "../data/test.faiss"
index = index_load(filepath, is_binary=True)

Prepare the search query

In [9]:
search_results = 10
query = [fen]

In [10]:
dist, reconstructed = index_query_positions(query, index, input_format='fen', output_format='board',
                                            num_results=search_results)


 (97,)


In [6]:
print(dist)

[[2 2 2 2 2 2 2 2 2 2]]


In [57]:
for i in range(len(reconstructed)):
    for j in range(search_results):
        print(f"Result {j} (best to worst)")
        print(reconstructed[i][j].fen())

Result 0 (best to worst)
rnbqkbnr/ppp1pppp/8/8/3p4/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Result 1 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 2 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 3 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 4 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 5 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 6 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1
Result 7 (best to worst)
rnbqkbnr/pppppppp/8/3P4/8/8/PPP1PPPP/RNBQKBNR w KQkq - 0 1
Result 8 (best to worst)
rnbqkb1r/pppppppp/8/8/4n3/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Result 9 (best to worst)
rnbqkbnr/pppppppp/8/4P3/8/8/PPPP1PPP/RNBQKBNR w KQkq - 0 1


In [59]:
from IPython.display import HTML
html = '''
<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous">
<table>
  <tr>
    <td>Your Query Position</td>
    <td>
      <select id="mySelect" onchange="myFunction()">
'''
for i in range(len(reconstructed)):
    for j in range(search_results):
        html += "<option value='"+reconstructed[i][j].fen()+"'>Position "+str(j)+"</option>"

html += '''
      </select>
    </td>
  </tr>
  <tr>
    <td><div id="query" style="width: 400px"></div></td>
    <td><div id="myBoard" style="width: 400px"></div></td>
  </tr>
</table>

<script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script>

<script>
var pos = document.getElementById("mySelect").value;
var board = Chessboard('myBoard',{showNotation: false});
var query = Chessboard('query',{
  position: 'r1bqkbnr/pppp1ppp/2n5/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R',
  showNotation: false
});

function myFunction() {
  var pos = document.getElementById("mySelect").value;
  console.log(pos);
  board.position(pos);
}
</script>
'''
HTML(html)

Your Query Position,Position 0Position 1Position 2Position 3Position 4Position 5Position 6Position 7Position 8Position 9
,


In [41]:
import chess
str(chess.Board()).split(' ')

['r',
 'n',
 'b',
 'q',
 'k',
 'b',
 'n',
 'r\np',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p',
 'p\n.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.\n.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.\n.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.\n.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.\nP',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P',
 'P\nR',
 'N',
 'B',
 'Q',
 'K',
 'B',
 'N',
 'R']